In [1]:
import os
os.chdir("Desktop")

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import time
import sys
import os
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pickle


firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('permissions.default.stylesheet', 2)
firefox_profile.set_preference('permissions.default.image', 2)
firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
# create driver
driver = webdriver.Firefox(firefox_profile=firefox_profile)

base_url = lambda hashtag : 'https://twitter.com/hashtag/{}?src=hash'.format(hashtag)
res=[]
hasht=[]
def get_soup(url):
    return BeautifulSoup( requests.get(url).content, 'lxml')

def crawl_page(url, n):
    # open url
    driver.get(url)
    # wait for page to load
    driver.implicitly_wait(15)
    # scroll for n seconds
    for i in range(n):
        elem = driver.find_element_by_tag_name('a')
        elem.send_keys(Keys.END)
        time.sleep(2)
        sys.stderr.write('\r{0}/{1} complete...'.format(i+1,n))
    # gather list items
    list_items = driver.find_elements_by_tag_name('ol')
    # get soup
    if list_items:
        soup = BeautifulSoup(list_items[0].get_attribute('innerHTML'),'lxml')
        return soup
    else:
        return False

def extract_tweet_ids(soup,hashtag):
    h=hashtag
    for i in soup.find_all('li', {"data-item-type":"tweet"}):
        try:
            text = (i.p.get_text())
        #print(text)
            res.append(text)
            hasht.append(h)
        except AttributeError:
            pass
            
        
            

    return res,hasht


        
        
        
if __name__ == '__main__':

    hashtags = ['mondaymotivation','bestfriends']
    n=50
    tweet_ids,hashstore=[],[]
    hashlist=[]
    for hashtag in hashtags:
        print('>> Crawling for #{}'.format(hashtag))
        soup = crawl_page(base_url(hashtag), n)
        if soup:
            tweet_ids,hashstore = extract_tweet_ids(soup,hashtag)
        
            print('>> Grabbed {0} tweets from {1}...'.format(len(tweet_ids),hashtag))
        else:
            pass
    
    t=[[i] for i in tweet_ids]
    df =  pd.DataFrame(t)
    df['hashtags'] = pd.Series(hashstore).values
    
    df.to_csv('outsample.csv', sep=',', header=['tweet_text','hastag'], index=None)

    
    driver.quit()

>> Crawling for #mondaymotivation


50/50 complete...

>> Grabbed 981 tweets from mondaymotivation...
>> Crawling for #bestfriends


50/50 complete...

>> Grabbed 1959 tweets from bestfriends...


In [5]:
os.getcwd()

'C:\\Users\\santh_000'